In [41]:
import cv2
import torch
import numpy as np
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image
from scipy.spatial.distance import cosine



In [42]:
# Initialize MTCNN for face detection and FaceNet for embedding extraction
mtcnn = MTCNN()
facenet = InceptionResnetV1(pretrained='vggface2').eval()


In [45]:

# Dictionary to store embeddings
known_faces = {}

image_path = r"C:\Users\Yathish\cv\person1.jpeg"   # Change this to actual image path
image_name = os.path.basename(image_path).split('.')[0]  # Extracts "person1" from "person1.jpeg"

# Load and detect face from the image
img = Image.open(image_path)
img_cropped = mtcnn(img)

if img_cropped is not None:
    img_cropped = img_cropped.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        embedding = facenet(img_cropped)  # Generate embedding

    # Store embedding using extracted name
    known_faces[image_name] = embedding.numpy()

    # Save embeddings to a file
    np.save("known_faces.npy", known_faces)
    print(f"Face embedding for {image_name} stored successfully!")
else:
    print("No face detected in the provided image.")



Face embedding for person1 stored successfully!


Before running the next cell, update the image path to the person you want to recognize. When that person appears on the webcam, the system will detect them and display "Hi [Person's Name]!" on the screen.

In [46]:

# Load stored embeddings
try:
    known_faces = np.load("known_faces.npy", allow_pickle=True).item()
    print("Loaded known faces:", known_faces.keys())
except FileNotFoundError:
    print("No stored faces found! Please store embeddings first.")
    known_faces = {}

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces
    boxes, _ = mtcnn.detect(frame)

    if boxes is not None:
        for box in boxes:
            x, y, x2, y2 = map(int, box)  # Convert bounding box coordinates to integers

            # Extract face ROI
            face_roi = frame[y:y2, x:x2]

            # Ensure the face ROI is valid before processing
            if face_roi.size == 0:
                continue

            face_roi = cv2.resize(face_roi, (160, 160))  # Resize for FaceNet

            # Convert to tensor for FaceNet
            face_roi = np.transpose(face_roi, (2, 0, 1))  # Change to (C, H, W)
            face_roi = torch.tensor(face_roi, dtype=torch.float32).unsqueeze(0) / 255.0  # Normalize

            # Generate face embedding
            with torch.no_grad():
                new_embedding = facenet(face_roi)

            # Convert embedding to NumPy array
            new_embedding = new_embedding.squeeze().numpy()

            # Function to recognize face
            def recognize_face(new_embedding, known_faces, threshold=0.6):
                min_distance = float("inf")
                best_match = "Unknown"

                for name, stored_embedding in known_faces.items():
                    stored_embedding = stored_embedding.squeeze()  # Ensure 1-D
                    distance = cosine(new_embedding, stored_embedding)

                    if distance < min_distance and distance < threshold:
                        min_distance = distance
                        best_match = name

                return best_match

            # Get recognized name
            recognized_name = recognize_face(new_embedding, known_faces)

            # Display name on screen
            if recognized_name != "Unknown":
                cv2.putText(frame, f"Hi {recognized_name}!", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            # Draw bounding box
            cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)

    # Show frame
    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Loaded known faces: dict_keys(['person1'])


## 📌 Previous Attempts
The cells below show earlier versions of the code, like  the progression.

In [7]:
import torch
from facenet_pytorch import InceptionResnetV1

model = InceptionResnetV1(pretrained='vggface2').eval()
print("FaceNet Model Loaded Successfully!")


100%|███████████████████████████████████████████████████████████████████████████████| 107M/107M [00:02<00:00, 48.3MB/s]


FaceNet Model Loaded Successfully!


In [35]:
from facenet_pytorch import MTCNN
from PIL import Image

# Initialize MTCNN 
mtcnn = MTCNN()

# Load an image
image_path = r"C:\Users\Yathish\Internship\cv\person1.jpeg"  # Change this to an actual image path
img = Image.open(image_path)

# Detect faces
boxes, probs = mtcnn.detect(img)

# Print detected face boxes and probabilities
print("Face Boxes:", boxes)
print("Probabilities:", probs)


Face Boxes: [[368.26373291015625 527.827880859375 1028.166748046875 1398.841064453125]]
Probabilities: [0.999993085861206]


In [22]:
import torch
from facenet_pytorch import InceptionResnetV1

# Load FaceNet model
model = InceptionResnetV1(pretrained='vggface2').eval()

# Ensure model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Extract face embedding
img_cropped = mtcnn(img)  
if img_cropped is not None:
    img_cropped = img_cropped.to(device)
    embedding = model(img_cropped.unsqueeze(0))  
    print("Face Embedding:", embedding.shape)
else:
    print("No face detected.")


Face Embedding: torch.Size([1, 512])


In [36]:
import torch
import numpy as np


known_faces = {}


name = "John_Doe"  

# Save the embedding 
known_faces[name] = embedding.detach().numpy()


np.save("known_faces.npy", known_faces)

print("Face embedding stored successfully!")


Face embedding stored successfully!


In [38]:

# Load stored embeddings
try:
    known_faces = np.load("known_faces.npy", allow_pickle=True).item()
    print("Loaded known faces:", known_faces.keys())
except FileNotFoundError:
    print("No known faces found! Please store embeddings first.")
    known_faces = {}


new_face_embedding = embedding.detach().numpy() 

# Compare with stored embeddings
def recognize_face(new_embedding, known_faces, threshold=0.6):
    if isinstance(new_embedding, torch.Tensor):
        new_embedding = new_embedding.squeeze().cpu().numpy()  
    else:
        new_embedding = new_embedding.squeeze()  

    min_distance = float("inf")
    best_match = None

    for name, stored_embedding in known_faces.items():
        stored_embedding = stored_embedding.squeeze()  # Ensure it's 1-D
        distance = cosine(new_embedding, stored_embedding)

        if distance < min_distance and distance < threshold:
            min_distance = distance
            best_match = name

    return best_match if best_match else "Unknown"



# Recognize the face
recognized_name = recognize_face(new_face_embedding, known_faces)
print(f"Recognized: {recognized_name}")


Loaded known faces: dict_keys(['John_Doe'])
Recognized: John_Doe


In [27]:
import cv2
import torch
import numpy as np
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image
from scipy.spatial.distance import cosine

# Initialize MTCNN
mtcnn = MTCNN()
facenet = InceptionResnetV1(pretrained='vggface2').eval()

# Dictionary to store embeddings
known_faces = {}


image_path = r"C:\Users\Yathish\cv\person1.jpeg" 
img = Image.open(image_path)

# Detect and extract face from image
img_cropped = mtcnn(img)

if img_cropped is not None:
    img_cropped = img_cropped.unsqueeze(0)  
    with torch.no_grad():
        embedding = facenet(img_cropped)  # Generate embedding

    # Store embedding with name "Yathish"
    known_faces["Yathish"] = embedding.numpy()

    
    np.save("known_faces.npy", known_faces)
    print("Face embedding for Yathish stored successfully!")
else:
    print("No face detected in the provided image.")
    exit()


try:
    known_faces = np.load("known_faces.npy", allow_pickle=True).item()
    print("Loaded known faces:", known_faces.keys())
except FileNotFoundError:
    print("No stored faces found! Please store embeddings first.")
    exit()

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces
    boxes, _ = mtcnn.detect(frame)

    if boxes is not None:
        for box in boxes:
            x, y, x2, y2 = map(int, box)  # Convert bounding box coordinates to integers

            # Ensure valid bounding box coordinates
            h, w, _ = frame.shape  # Get frame dimensions
            x = max(0, x)
            y = max(0, y)
            x2 = min(w, x2)
            y2 = min(h, y2)

            # Extract face ROI safely
            face_roi = frame[y:y2, x:x2]

            
            if face_roi.size == 0:
                continue  

            face_roi = cv2.resize(face_roi, (160, 160))  # Resize for FaceNet

            # Convert to tensor for FaceNet
            face_roi = np.transpose(face_roi, (2, 0, 1))  # Change to (C, H, W)
            face_roi = torch.tensor(face_roi, dtype=torch.float32).unsqueeze(0) / 255.0  # Normalize

            # Generate face embedding
            with torch.no_grad():
                new_embedding = facenet(face_roi)

            # Convert embedding to NumPy array
            new_embedding = new_embedding.squeeze().numpy()

            # Recognize face
            def recognize_face(new_embedding, known_faces, threshold=0.6):
                min_distance = float("inf")
                best_match = "Unknown"

                for name, stored_embedding in known_faces.items():
                    stored_embedding = stored_embedding.squeeze()  # Ensure 1-D
                    distance = cosine(new_embedding, stored_embedding)

                    if distance < min_distance and distance < threshold:
                        min_distance = distance
                        best_match = name

                return best_match

            # Get recognized name
            recognized_name = recognize_face(new_embedding, known_faces)

            # Display name on screen
            if recognized_name == "Yathish":
                cv2.putText(frame, "Hi Yathish!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            # Draw bounding box
            cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)

    # Show frame
    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Face embedding for Yathish stored successfully!
Loaded known faces: dict_keys(['Yathish'])
